# EBITDA Weight S&P 500 Index Fund

## Summary:
the S&P 500 is the worlds biggest stock market index and the index fund that follows the S&P has $250 billion under management.


The goal of this projects is to weight the investments in each stock within the S&P based on there EBITDA (Earnings Before Interest, Taxes, Depreciation, and Amortization) project was inspired by this video https://www.youtube.com/watch?v=xfzGZB4HhEE&t=5055s

## Imports

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python

## Get a list of S&P 500 stocks
I have a list of S&P stocks in the repo called sp_500_stocks.csv

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [4]:
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Use the IEX API

here is a link to the API https://iexcloud.io/docs/api/#quote we will need the stock price for each stock as well as the EBITDA. to get this we will have to make two different calls to the API.


In [11]:
# Stock Price
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448' # not unique, can be used for free API
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': ')E SASCL/TT LKGR(MGAE NAOAQSNBLDE',
 'calculationPrice': 'tops',
 'open': None,
 'openTime': None,
 'openSource': 'liioffca',
 'close': None,
 'closeTime': None,
 'closeSource': 'ilafifco',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 128.63,
 'latestSource': 'IEX real time price',
 'latestTime': '1:46:29 PM',
 'latestUpdate': 1613061544763,
 'latestVolume': None,
 'iexRealtimePrice': 129.76,
 'iexRealtimeSize': 102,
 'iexLastUpdated': 1626243115625,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 132.13,
 'previousVolume': 100753553,
 'change': 0.69,
 'changePercent': 0.00531,
 'volume': None,
 'iexMarketPercent': 0.012941206305403056,
 'iexVolume': 710

In [12]:
data['latestPrice']

128.63

In [13]:
# EBITDA
api_url = f'https://sandbox.iexapis.com/stable//stock/{symbol}/advanced-stats?token={IEX_CLOUD_API_TOKEN}'
data_EBITDA = requests.get(api_url).json()
data_EBITDA

{'beta': 1.1586392259075027,
 'totalCash': 92572216828,
 'currentDebt': 106565795839,
 'revenue': 281751012104,
 'grossProfit': 108625489059,
 'totalRevenue': 283948983392,
 'EBITDA': 77595540199,
 'revenuePerShare': 16.4,
 'revenuePerEmployee': 2063339.18,
 'debtToEquity': 5.102071685088987,
 'profitMargin': 0.2113805857378388,
 'enterpriseValue': 2250443005163,
 'enterpriseValueToRevenue': 8.24,
 'priceToSales': 8.11,
 'priceToBook': 34.55,
 'forwardPERatio': 32.3292228010501,
 'pegRatio': 374.5703174017598,
 'peHigh': 41.05154547246343,
 'peLow': 16.81231988923012,
 'week52highDate': '2020-08-25',
 'week52lowDate': '2020-03-19',
 'putCallRatio': 0.3364093212315464,
 'companyName': 'Apple Inc',
 'marketcap': 2203156927420,
 'week52high': 136.74,
 'week52low': 58.23,
 'week52change': 0.8726364233410062,
 'sharesOutstanding': 17546069622,
 'float': None,
 'avg10Volume': 104991349,
 'avg30Volume': 99521804,
 'day200MovingAvg': 111.14,
 'day50MovingAvg': 122.3,
 'employees': None,
 'ttmE

In [15]:
data_EBITDA['EBITDA']

77595540199

## Make an empty data frame


In [16]:
my_columns = ['Ticker', 'Price','EBITDA', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,EBITDA,Number Of Shares to Buy


In [17]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data_EBITDA['EBITDA'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

,Ticker,Price,EBITDA,Number Of Shares to Buy
0,AAPL,128.63,77595540199,N/A


## Batch API call
instead of calling the API 500 times you can make calls to the api of at max 100 stocks and this will only need to make 5 calls to the api and will be much faster (well technically it will be 10 calls because need to make two different calls to the api for ebitda and stock price.

In [18]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [24]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    batch_api_call_url_ebitda = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data_ebitda = requests.get(batch_api_call_url_ebitda).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data_ebitda[symbol]['advanced-stats']['EBITDA'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,EBITDA,Number Of Shares to Buy
0,A,124.550,1129402388,N/A
1,AAL,17.253,-5728534410,N/A
2,AAP,160.940,1011957279,N/A
3,AAPL,130.985,80051206099,N/A
4,ABBV,105.400,17212540887,N/A
...,...,...,...,...
500,YUM,110.790,1801202676,N/A
501,ZBH,152.240,1490087329,N/A
502,ZBRA,382.490,725656416,N/A
503,ZION,42.320,696213425,N/A


## Data Cleaning
going to remove EBIDTAs that are negative because i only want to buy stocks and not short stocks

In [27]:
final_dataframe = final_dataframe[final_dataframe['EBITDA']>0]

## Calculate the number of shares to buy

In [29]:
portfolio_size = 100000

ebitda_weights = final_dataframe['EBITDA'] / sum(final_dataframe['EBITDA'])

In [30]:
ebitda_weights[:5]

0    0.000530354
2    0.000475204
3      0.0375911
4     0.00808281
5     0.00108514
Name: EBITDA, dtype: object

In [40]:
position_size = portfolio_size * ebitda_weights

In [42]:
position_size // final_dataframe['Price']

0       0
2       0
3      28
4       7
5       1
       ..
500     0
501     0
502     0
503     0
504     0
Length: 473, dtype: object

In [44]:
number_of_shares = (position_size // final_dataframe['Price']).values

In [47]:
final_dataframe['Number Of Shares to Buy'] = number_of_shares

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
final_dataframe

,Ticker,Price,EBITDA,Number Of Shares to Buy
0,A,124.550,1129402388,0
2,AAP,160.940,1011957279,0
3,AAPL,130.985,80051206099,28
4,ABBV,105.400,17212540887,7
5,ABC,98.860,2310837584,1
...,...,...,...,...
500,YUM,110.790,1801202676,0
501,ZBH,152.240,1490087329,0
502,ZBRA,382.490,725656416,0
503,ZION,42.320,696213425,0


In [53]:

final_dataframe.sort_values('Number Of Shares to Buy',ascending=False)[:10]

,Ticker,Price,EBITDA,Number Of Shares to Buy
429,T,30.770,59049880203,90
56,BAC,30.140,22886284535,35
475,VZ,62.380,46441585175,34
174,F,9.420,6724434403,33
496,XOM,44.477,27537907517,29
99,CMCSA,51.549,32153377795,29
3,AAPL,130.985,80051206099,28
242,INTC,51.400,25442190087,23
198,GE,11.133,5378877850,22
365,PFE,38.920,17734810193,21
